# Selenium

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

In [3]:
url = 'https://svo2.cab.inta-csic.es/theory/newov2/index.php?models=bt-settl'

driver.get(url)

In [26]:
driver.find_elements(By.CLASS_NAME, 'mdls2_modtit')[0].text

'AMES-Cond 2000'

# Beautiful Soup

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_soup(url, verify=False):

    site = requests.get(url, verify=verify)
    
    return BeautifulSoup(site.content, 'html.parser')

In [3]:
url = "https://svo2.cab.inta-csic.es/theory/newov2/"
soup = get_soup(url)

In [4]:
def get_model_url(soup, model='BT-Settl'):

    model_entries = soup.find_all('td', class_='mdls2_modtit')

    # Search for the model
    model_href = None
    for entry in model_entries:
        b_tag = entry.find('b')
        if b_tag and model in b_tag.text:
            a_tag = b_tag.find('a')
            if a_tag and 'href' in a_tag.attrs:
                model_href = a_tag['href']
                break
    
    # Print the result
    if model_href:
        
        # Construct full URL if needed (if the href is relative)
        if not model_href.startswith('http'):
            full_url = url + model_href.split("newov2/")[1]
            return full_url
    else:
        print(f"{model} model not found in the page.")
        return

In [5]:
new_url = get_model_url(soup)
new_url

'https://svo2.cab.inta-csic.es/theory/newov2/index.php?models=bt-settl'

In [6]:
model_soup = get_soup(new_url)

In [7]:
def submit_params(url, model='BT-Settl', teff_min=5000, teff_max=5100, logg_min=4, logg_max=4.5, meta_min=0, meta_max=0.5):

    model = model.lower()

    # Prepare the form data with your desired parameters
    form_data = {
        'models': ','+model,
        'oby': '',
        'odesc': '',
        'sbut': '',
        # Set Teff min and max
        f'params[{model}][teff][min]': str(teff_min),
        f'params[{model}][teff][max]': str(teff_max),
        # Set logg min and max
        f'params[{model}][logg][min]': str(logg_min),
        f'params[{model}][logg][max]': str(logg_max),
        # Set metallicity min and max
        f'params[{model}][meta][min]': str(meta_min),
        f'params[{model}][meta][max]': str(meta_max),
        # Show all results
        'nres': 'all',
        # Submit button
        'boton': 'Search',
        # Selected model
        'reqmodels[]': model
    }
    
    # Now follow the redirect if needed to get the actual content
    response = requests.post(url, data=form_data, allow_redirects=True, verify=False)

    if response.status_code == 200:

        print('Form submitted successfully!')
        return BeautifulSoup(response.text, 'html.parser')

    else:
        return

In [8]:
# Example usage
form_soup = submit_params(
    url = new_url,
    teff_min=5000, 
    teff_max=5100, 
    logg_min=4, 
    logg_max=4.5, 
    meta_min=0, 
    meta_max=0.5
)

Form submitted successfully!


In [9]:
def make_request(url, method='get', data=None):
    """Make a request with error handling"""
    try:
        if method.lower() == 'post':
            response = requests.post(url, data=data, verify=False, allow_redirects=True)
        else:
            response = requests.get(url, verify=False, allow_redirects=True)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [13]:
selected_models = []
rows = form_soup.select('table table tr')[1:]  # Skip header row

for i, row in enumerate(rows):
    cells = row.find_all('td')
    if len(cells) < 10:  # Skip non-data rows
        continue
    
    model = cells[0].get_text(strip=True)
    alpha = cells[4].get_text(strip=True)
    
    if model == "BT-Settl" and alpha == "0":
        ascii_link = cells[7].find('a')
        if ascii_link and 'href' in ascii_link.attrs:
            selected_models.append({
                'index': i,
                'model': model,
                'teff': cells[1].get_text(strip=True),
                'logg': cells[2].get_text(strip=True),
                'meta': cells[3].get_text(strip=True),
                'alpha': alpha,
                'ascii_link': url + ascii_link['href']
            })

if not selected_models:
    print("No matching models found with Alpha=0")
    exit()

original_form = form_soup.find('form', {'name': 'myform'})
form_action = url + original_form['action'].split("newov2/")[1]
original_data = {}

# Get hidden inputs and select values
for input_tag in original_form.find_all(['input', 'select']):
    if input_tag.name == 'input' and input_tag.get('type') == 'hidden':
        original_data[input_tag['name']] = input_tag.get('value', '')
    elif input_tag.name == 'select':
        selected_option = input_tag.find('option', selected=True)
        if selected_option:
            original_data[input_tag['name']] = selected_option.get('value', '')

# Step 2: Prepare data for retrieval (include original form data)
retrieve_data = original_data.copy()  # Keep all original selections

# Mark ASCII checkboxes for selected models
for model in selected_models:
    retrieve_data[f'txt[{model["index"]}]'] = 'on'

retrieve_data['boton'] = 'Retrieve'

# Step 7: Submit the retrieval request
print(f"Submitting retrieval request for {len(selected_models)} models...")
response = make_request(form_action, 'post', retrieve_data)
if not response:
    exit()

# # Step 8: Wait for server to prepare the download

# processing_delay = len(selected_models) * 3 # 3 seconds per model

# print(f"Waiting {processing_delay} seconds for server to prepare download...")
# time.sleep(processing_delay)

# # Step 9: Check for download link (you might need to reload the page)
# print("Checking for download link...")
# response = make_request(form_action)
# if not response:
#     exit()

soup = BeautifulSoup(response.text, 'html.parser')
download_link = soup.find('a', string=lambda t: t and 'Download tar file' in t)

if download_link and 'href' in download_link.attrs:
    download_url = download_link['href']
    print(f"Download URL found: {download_url}")
    
    # # Step 10: Download the file
    # print("Starting download...")
    # download_response = make_request(download_url)
    # if download_response:
    #     filename = f"bt_settl_models_{int(time.time())}.tar"
    #     with open(filename, 'wb') as f:
    #         f.write(download_response.content)
    #     print(f"Download completed! Saved as {filename}")
    # else:
    #     print("Failed to download file")
else:
    print("No download link found. Possible reasons:")
    print("- The server needs more time to prepare the files")
    print("- There might be an error in the request")
    print("- The page structure might have changed")

# Print selected models for verification
print("\nSelected models summary:")
for model in selected_models:
    print(f"Model: {model['model']}, Teff: {model['teff']}, Logg: {model['logg']}, "
          f"Metallicity: {model['meta']}, Alpha: {model['alpha']}")

Submitting retrieval request for 12 models...
Download URL found: https://svo2.cab.inta-csic.es//theory/newov2/temp/models/tar/models_1745273472.tgz

Selected models summary:
Model: BT-Settl, Teff: 5000, Logg: 4, Metallicity: 0, Alpha: 0
Model: BT-Settl, Teff: 5000, Logg: 4.5, Metallicity: 0.5, Alpha: 0
Model: BT-Settl, Teff: 5000, Logg: 4.5, Metallicity: 0.3, Alpha: 0
Model: BT-Settl, Teff: 5000, Logg: 4.5, Metallicity: 0, Alpha: 0
Model: BT-Settl, Teff: 5000, Logg: 4, Metallicity: 0.5, Alpha: 0
Model: BT-Settl, Teff: 5000, Logg: 4, Metallicity: 0.3, Alpha: 0
Model: BT-Settl, Teff: 5100, Logg: 4.5, Metallicity: 0.3, Alpha: 0
Model: BT-Settl, Teff: 5100, Logg: 4.5, Metallicity: 0, Alpha: 0
Model: BT-Settl, Teff: 5100, Logg: 4, Metallicity: 0.5, Alpha: 0
Model: BT-Settl, Teff: 5100, Logg: 4, Metallicity: 0.3, Alpha: 0
Model: BT-Settl, Teff: 5100, Logg: 4, Metallicity: 0, Alpha: 0
Model: BT-Settl, Teff: 5100, Logg: 4.5, Metallicity: 0.5, Alpha: 0
